In [ ]:
import pandas as pd
import datetime

In [ ]:
date_str = datetime.datetime.now().strftime("%Y-%m-%d")

js = pd.read_json('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_calendar_' + date_str + '.json')

js.head()

In [ ]:
import datetime
import re
import lxml
from bs4 import BeautifulSoup
import json
#import bs4 as BeautifulSoup
from calendar import monthrange

d = datetime.date.today()
mr = monthrange(d.year, d.month )
cur_month = d.month

first_page = True



fp = open('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_processed_calendar_' + run_ts + '.json', 'w')


for index, row in js.iterrows():   
    
    all_months = {}
    
    cur_month = d.month

    # Set all the six months
    for i in range(cur_month,cur_month+6):

        mr = monthrange( d.year, i )

        max_day = mr[1]

        all_months[i] = dict.fromkeys(range(1,max_day+1),'')

    
    raw_cal = "<html><body><table>" + row['calendar'] + "</table></body></html>"
    
    soup = BeautifulSoup(raw_cal, 'lxml') # Parse the HTML as a string
    soup_tables = soup.find_all('tbody')# Grab the first table

    for month_tb in soup_tables:
 
        table = soup.find("table")
        
        cells = table.findAll("td")
            
        for td in cells:

            try:
                day_val = int(td.text)
                
                #----------------------------------------------------------
                # Check if DEP and ARR can both be on the same day
                # for consecutive bookings?

                # If its an unavailable date then mark: 
                m = re.search('c-calendar--unavailable', str(td))

                if m is not None:
                    all_months[cur_month][day_val] = 'UVL' # Unavailable day

                # If its an unavailable date then mark: 
                m = re.search('c-calendar--arrival-day', str(td))

                if m is not None:
                    all_months[cur_month][day_val] = 'ARR' # Arrival day

                # If its an unavailable date then mark: 
                m = re.search('c-calendar--departure-day', str(td))

                if m is not None:
                    all_months[cur_month][day_val] = 'DEP' # Departure day

                m = re.search('c-calendar--available', str(td))
                
                if m is not None:
                    all_months[cur_month][day_val] = 'AVL' # Available day


            except ValueError:
                day_val = None

        # Move to the next month
        cur_month += 1

    # Print the table to check
    #print(all_months)
    
    cal_details = {}
    cal_details['property_id'] = row['property_id']
    cal_details['calendar'] = all_months
    
    # Write the data to JSON
    json.dump(cal_details, fp)
    
    if first_page is True:
        fp.write('[\n')
        first_page = False
    else:
        fp.write('\n,')

# Close out the JSON format
fp.write('\n]')    

# Tidy up
fp.close()

In [ ]:
run_ts = '2018-02-26'
print(run_ts)
p = pd.read_json('/Users/taj/GitHub/scraping/chrome_stayz_calendar/WebData/stayz_processed_calendar' + run_ts + '.json')
p.head()

In [ ]:
c = p['calendar']
c.head()